<a href="https://colab.research.google.com/github/srilamaiti/ml_works/blob/main/us_prod_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torchvision
!pip install torchaudio
!pip install wordcloud
!pip install TextBlob
!pip install spacy
!pip install textstat
!pip install pyldavis
!pip install flair
!pip install transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
from wordcloud import WordCloud, STOPWORDS
from textblob import TextBlob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import os
import nltk
from collections import defaultdict
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pyLDAvis
import collections
from nltk.stem import PorterStemmer
import gensim
import re
import pyLDAvis.gensim_models as gensimvis
nltk.download('punkt')
nltk.download('wordnet')
#from keybert import KeyBERT
import spacy
import plotly.express as px
import string
from nltk.tokenize import sent_tokenize
from flair.data import Sentence
from flair.nn import Classifier

pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
def get_sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

#sentiment_pipeline = pipeline("sentiment-analysis")
def polarity(text):
    return TextBlob(text).sentiment.polarity

def subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

'''
def sentiment(text):
    try:
        return sentiment_pipeline(subset)['label'],  sentiment_pipeline(subset)['score']
    except:
        -99, -99
'''

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        max_words=200,
        max_font_size=40,
        scale=1,
        random_state=1
).generate(" ".join(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

def tokenize(text):
    """ basic tokenize method with word character, non word character and digits """
    text = re.sub(r" +", " ", str(text))
    text = re.split(r"(\d+|[a-zA-ZğüşıöçĞÜŞİÖÇ]+|\W)", text)
    text = list(filter(lambda x: x != '' and x != ' ', text))
    sent_tokenized = ' '.join(text)
    return sent_tokenized

regex = re.compile('[%s]' % re.escape(string.punctuation))

def remove_punct(text):
    text = regex.sub(" ", text)
    return text

def get_top_ngram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx])
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]

def plot_dist(df, col):
    sns.distplot(df[col], hist=True, kde=True,
             bins=int(180/5), color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

def get_sentiment_raw(review_text):
    sentence = Sentence(review_text)

    # load the NER tagger
    tagger = Classifier.load('sentiment')

    # run NER over sentence
    tagger.predict(sentence)

    return str(sentence)

def get_sentiment_val(review_text):
    sentence = Sentence(review_text)

    # load the NER tagger
    tagger = Classifier.load('sentiment')

    # run NER over sentence
    tagger.predict(sentence)

    return str(sentence).split(" → ")[1].split(" ")[0]

def get_sentiment_score(review_text):
    sentence = Sentence(review_text)

    # load the NER tagger
    tagger = Classifier.load('sentiment')

    # run NER over sentence
    tagger.predict(sentence)

    return float(str(sentence).split(" → ")[1].split(" ")[1].replace("(","").replace(")",""))


In [11]:
df_review = pd.read_csv("review.csv")
df_review['REVIEW_TEXT'] = df_review['REVIEW_TEXT'].apply(lambda x : str(x))
df_review['REVIEW_TEXT'] = df_review['REVIEW_TEXT'].apply(lambda x : x.strip())
df_review['REVIEW_TEXT_LEN'] = df_review['REVIEW_TEXT'].apply(lambda x : len(str(x)))

In [12]:
df_review.shape

(331271, 6)

In [13]:
df_review.head()

,BRAND,REVIEW_TEXT,REGIONLANGUAGE,PRODUCTRATING,PRODUCTREVIEWCOUNT,REVIEW_TEXT_LEN
0,baseus,"Product is really good, easy to use, great for...",EN,4.2,158.0,75
1,baseus,"Wish it was a little stronger, but it is actua...",EN,4.2,158.0,501
2,shark,I’m obsessed With this vacuum!! I use it daily...,EN,4.4,73.0,387
3,baseus,"Used for one car, great for light cleaning and...",EN,4.2,158.0,171
4,baseus,Whoever designed this is a genius. It’s qualit...,EN,4.2,158.0,533


In [14]:
df_review = df_review[['BRAND', 'REVIEW_TEXT', 'REVIEW_TEXT_LEN', 'PRODUCTRATING']].drop_duplicates()

In [15]:
df_review.shape

(293775, 4)

In [16]:
df_review['REVIEW_TEXT'] = df_review["REVIEW_TEXT"].replace(re.compile(r'[\n\r\t]'), ' ', regex=True)
df_review['REVIEW_WORD_COUNT'] = df_review['REVIEW_TEXT'].apply(lambda x : len(str(x).split()))
df_review = df_review[df_review['REVIEW_WORD_COUNT'] > 1]
df_review['REVIEW_SENTENCE_COUNT'] = df_review['REVIEW_TEXT'].apply(lambda x : len(sent_tokenize(x)))
df_review['REVIEW_TEXT'] = df_review["REVIEW_TEXT"].apply(lambda x : remove_punct(x))
df_review['REVIEW_POLARITY'] = df_review["REVIEW_TEXT"].apply(lambda x : polarity(x))
df_review['REVIEW_SUBJECTIVITY'] = df_review["REVIEW_TEXT"].apply(lambda x : subjectivity(x))
df_review.head()

,BRAND,REVIEW_TEXT,REVIEW_TEXT_LEN,PRODUCTRATING,REVIEW_WORD_COUNT,REVIEW_SENTENCE_COUNT,REVIEW_POLARITY,REVIEW_SUBJECTIVITY
0,baseus,Product is really good easy to use great for...,75,4.2,14,1,0.644444,0.727778
1,baseus,Wish it was a little stronger but it is actua...,501,4.2,103,5,0.040104,0.513542
2,shark,I’m obsessed With this vacuum I use it daily...,387,4.4,80,4,0.169630,0.517037
3,baseus,Used for one car great for light cleaning and...,171,4.2,32,3,0.306667,0.450000
4,baseus,Whoever designed this is a genius It’s qualit...,533,4.2,102,7,0.100000,0.537500


In [17]:
df_review_count = df_review.groupby(['BRAND']).agg({"REVIEW_TEXT": np.size}).reset_index().sort_values(by = ['REVIEW_TEXT'] )
df_review_count.columns = ['BRAND', 'REVIEW_COUNT']
df_review_count.sort_values(by = ['REVIEW_COUNT'], ascending=False ).head(5)

,BRAND,REVIEW_COUNT
8,black+decker,61116
7,bissell,39588
63,vaclife,28922
54,shark,24451
17,dirt devil,12201


In [18]:
top_5_brands = df_review_count.sort_values(by = ['REVIEW_COUNT'], ascending=False ).head(5)['BRAND'].values.tolist()
top_5_brands

['black+decker', 'bissell', 'vaclife', 'shark', 'dirt devil']

In [19]:
len(df_review[['BRAND']].drop_duplicates())

73

In [20]:
bins = [0, 500, 1000, 5000, 10000, 25000, 50000, 75000, 1000000]
groups = df_review_count.groupby(pd.cut(df_review_count.REVIEW_COUNT, bins))
groups.BRAND.count()

REVIEW_COUNT
(0, 500]             5
(500, 1000]         23
(1000, 5000]        37
(5000, 10000]        2
(10000, 25000]       3
(25000, 50000]       2
(50000, 75000]       1
(75000, 1000000]     0
Name: BRAND, dtype: int64

In [24]:
df_review_stats = df_review.groupby(['BRAND']).agg({"PRODUCTRATING": [np.min, np.size]}).reset_index()
df_review_stats.columns = ['brand', 'avg_rating', 'rating_count']
df_review_stats

,brand,avg_rating,rating_count
0,absob,3.8,670
1,amazon renewed,2.5,1841
2,aonesy,2.8,813
3,as awesling,4.3,1451
4,auloea,3.3,448
...,...,...,...
68,whall,2.5,1896
69,worx,4.4,909
70,xiaomi,3.8,1240
71,xrexs,4.2,552


In [ ]:
estimate_list = []
for brand in df_review['BRAND'].drop_duplicates().values.tolist():
    for i in range(10000):
        temp_df = df_review[df_review.BRAND == brand][['PRODUCTRATING']]
        estimate = temp_df['PRODUCTRATING'].sample(n = 100, replace = True, random_state = 1234).mean()
        estimate_list.append((brand, estimate))
df_estimate = pd.DataFrame(estimate_list)
df_estimate.columns = ['BRAND', 'ESTIMATED_SAMPLE_MEAM']
df_estimate

In [14]:
brand_review_list = []

In [15]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
df_review_sentiment = pd.DataFrame()
temp_df = pd.DataFrame()
for brand in top_5_brands:#df_review['BRAND'].drop_duplicates().values.tolist():
    temp_df = df_review[df_review.BRAND == brand][['BRAND','REVIEW_TEXT']]
    print(f"Brand: {brand}")
    print(f"Number of reviews: {len(temp_df)}")
    for i in list(list(range(0, len(temp_df), 1000))):
        print(i)
        temp_df['REVIEW_TEXT_SENTIMENT'] = temp_df.iloc[i : i + 1000, :]['REVIEW_TEXT'].apply(lambda x: get_sentiment_score(x[:512]))
        df_review_sentiment = pd.concat([df_review_sentiment, temp_df], ignore_index=True)
    temp_df = pd.DataFrame()

Brand: black+decker
Number of reviews: 57770
0


In [ ]:
df_review_sentiment

In [ ]:
#df_review['REVIEW_TEXT_SENTIMENT'] = df_review['REVIEW_TEXT'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
tokens = tokenizer.encode("good 😄", return_tensors='pt')
result = model(tokens)
int(torch.argmax(result.logits))+1

4